In [1]:
import numpy as np
import time
import os
import shutil

import random
import pandas as pd

from pylab import *
import matplotlib.pyplot as plt
# from matplotlib_venn import *
%matplotlib inline
import seaborn as sns

from scipy import stats
from numpy.random import *

# from lifelines import KaplanMeierFitter
# from lifelines.statistics import logrank_test

import statsmodels.stats.multitest 



In [2]:

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss
from sklearn.metrics import make_scorer

from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectPercentile, f_classif, chi2
from sklearn.pipeline import make_pipeline

from sklearn.utils.class_weight import compute_sample_weight

from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import ExtraTreesClassifier
# from lightgbm import lightgbmClassifier
import lightgbm
from lightgbm import LGBMClassifier

In [3]:
sys.path.append('../pyvenn')
import venn
# cd ../ipynb/

ModuleNotFoundError: No module named 'venn'

In [4]:
def time_output(elapsed_time):
    hour = int(elapsed_time/3600)
    elapsed_time -= hour*3600
    minute = int(elapsed_time/60)
    elapsed_time -= minute*60
    second = int(elapsed_time)
    return(str(hour)+":"+str(minute)+":"+str(second))
#     print (str(hour)+":"+str(minute)+":"+str(second))


In [5]:
# import warnings
# warnings.filterwarnings('ignore')


In [6]:
# wd = "/Users/rpsugino/Dropbox/shared/NB450K_draft/20201002_K27ac/"
datadir = "../data/"
outdir = "../result/class_prediction/"

if os.path.isdir("../result/")==False:
    os.mkdir("../result/")
if os.path.isdir(outdir)==False:
    os.mkdir(outdir)

# my class

In [7]:
df_samples = pd.read_csv(datadir+"4data_clin_age.csv", index_col = 0)

for i, items in df_samples.iterrows():
    df_samples.at[i, "class"] = "D"
    if df_samples.at[i, "MYCN"] == "Y":
        df_samples.at[i, "class"] = "A"
    elif df_samples.at[i, "stage"] == "4":
        df_samples.at[i, "class"] = "B"
    elif df_samples.at[i, "stage"] == "4s":
        df_samples.at[i, "class"] = "C"
    # elif df_samples.at[i, "stage"] == "3":
    #     df_samples.at[i, "class"] = "3"
        
#     event/not 	relapse/otherevent/not 	alive/death
df_samples

,class,dataset,stage,MYCN,event/not,relapse/otherevent/not,alive/death,Event_Free_Survival_Time_in_Days,Overall_Survival_Time_in_Days,FirstEvent,diag_class,diag_age
ID,,,,,,,,,,,,
TARGET-30-PAIFXV-01A-01D,B,Target,4,N,Event,Event,D,630.0,1114.0,death of disease,1,2004
TARGET-30-PAIPGU-01A-01D,B,Target,4,N,Event,Death,D,436.0,436.0,death of disease,1,898
TARGET-30-PAISNS-01A-01D,B,Target,4,N,Event,Event,D,410.0,421.0,death of disease,1,1070
TARGET-30-PAITCI-01A-01D,B,Target,4,N,Event,Event,D,232.0,324.0,death of disease,1,728
TARGET-30-PAIVHE-01A-01D,B,Target,4,N,Event,Event,D,672.0,1605.0,death of disease,1,1123
...,...,...,...,...,...,...,...,...,...,...,...,...
GSM3407024,D,Ackermann,1,N,Event,relapse/progression,D,NaN,NaN,relapse/progression,0,0
GSM3407025,B,Ackermann,4,N,Not,no event,A,268.0,268.0,no event,1,2786
GSM3407026,D,Ackermann,3,N,Not,no event,A,NaN,NaN,no event,0,323


In [8]:
df_beta = pd.read_csv(datadir+"4data_all.tsv.gz", index_col=0)
df_beta = df_beta.dropna(how='any')
df_beta = df_beta.loc[df_beta.index[:10000]]
df_beta

,B_NB49_TG1,GSM1897038,TARGET-30-PARSRJ-01A-01D,CNB078_TG1_GD,CNB051_TG1,TARGET-30-PARRBU-01A-01D,TARGET-30-PARABN-01A-01D,CNB004_TG1,CNB033_TG1,GSM1897034,...,CNB043_TG1,GSM3406979,TARGET-30-PALEAC-01A-01D,B_NB25_TG1,GSM1897015,GSM1896980,B_NB47_TG1,GSM1897007,GSM1896992,TARGET-30-PAPVEB-01A-01D
ID_REF,,,,,,,,,,,,,,,,,,,,,
cg07881041,0.858244,0.706292,0.690741,0.794879,0.530863,0.737606,0.654847,0.773866,0.713572,0.663361,...,0.492197,0.670273,0.841101,0.847665,0.841114,0.763605,0.782868,0.659464,0.865294,0.696973
cg18478105,0.024347,0.017197,0.015366,0.032762,0.041506,0.011863,0.014571,0.019259,0.024432,0.007014,...,0.029396,0.050281,0.010114,0.036258,0.049479,0.044862,0.057535,0.081491,0.042719,0.017166
cg23229610,0.892562,0.778395,0.839263,0.830428,0.826637,0.849014,0.915327,0.831216,0.832141,0.912968,...,0.827100,0.839426,0.891314,0.840331,0.841796,0.765193,0.884502,0.871245,0.858777,0.912945
cg03513874,0.861707,0.575521,0.464077,0.205124,0.562217,0.327784,0.189340,0.631601,0.602789,0.848280,...,0.162827,0.568663,0.547845,0.838362,0.571128,0.606321,0.830910,0.372907,0.590636,0.901829
cg09835024,0.042063,0.014597,0.051100,0.043786,0.046506,0.017642,0.047186,0.036971,0.016751,0.002093,...,0.050351,0.038511,0.035308,0.031027,0.037183,0.051001,0.055149,0.049585,0.063245,0.036969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
cg04868734,0.914407,0.970958,0.847044,0.885893,0.897050,0.798319,0.922166,0.897573,0.896801,0.962188,...,0.893063,0.930035,0.920204,0.896157,0.906637,0.803575,0.932658,0.878059,0.887674,0.931263
cg14069736,0.008021,0.002859,0.003107,0.016160,0.002570,0.000000,0.000000,0.007576,0.003639,0.001775,...,0.003308,0.018612,0.008891,0.001375,0.003634,0.009065,0.010541,0.006893,0.004555,0.007444
cg03760226,0.578077,0.899564,0.693539,0.777359,0.531190,0.946330,0.859756,0.512642,0.520794,0.975426,...,0.892428,0.444773,0.981719,0.939405,0.926959,0.883715,0.993210,0.979248,0.706682,0.988036


In [9]:
it = 10

# 1000 times random forest 

In [10]:
%%time

start = time.time()

df_res = pd.DataFrame()
df_imp = pd.DataFrame(index=df_beta.index)

for i in range(it):
    print(i, time.time()-start)

    (X_train, X_test ,y_train, y_test) = train_test_split(df_beta.transpose(), df_samples["class"], test_size = 0.3)
    a = RandomForestClassifier(n_estimators=1000, max_depth=4, n_jobs=10, class_weight='balanced')

    selector = SelectPercentile(score_func=chi2, percentile=10) 
    selector.fit(X_train,y_train)
    X_selected = selector.transform(X_train)
    a = RandomForestClassifier(n_estimators=1000, max_depth=5,n_jobs=-1, class_weight='balanced')
    a.fit(X_selected,y_train)

    classes = sorted(df_samples["class"].unique())
    
    df_tmp = pd.DataFrame(a.feature_importances_, columns=[i],index=selector.transform(pd.DataFrame(df_beta.index).T)[0])
    df_imp = df_imp.join(df_tmp)
    
    pred_list = a.predict(selector.transform(X_test))
    prec_tmp,reca_tmp,f1_tmp,support_tmp = precision_recall_fscore_support(y_test, pred_list, average=None, labels=classes)
    
    df_res.at[i, "mean_score"] = a.score(selector.transform(X_test), y_test)
    for j in range(len(classes)):
        df_res.at[i, classes[j]+"_precision"] = prec_tmp[j]
        df_res.at[i, classes[j]+"_recall"] = reca_tmp[j]
        df_res.at[i, classes[j]+"_f1"] = f1_tmp[j]
        df_res.at[i, classes[j]+"_support"] = support_tmp[j]
df_res.to_csv(outdir+"RF.csv")
df_imp.to_csv(outdir+"RF_imp.csv")

0 0.00044798851013183594


/opt/anaconda3/envs/NB_ML0710/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but SelectPercentile was fitted with feature names
  warnings.warn(


1 2.7981691360473633


/opt/anaconda3/envs/NB_ML0710/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but SelectPercentile was fitted with feature names
  warnings.warn(


2 5.557663917541504


...

8 21.888412952423096


/opt/anaconda3/envs/NB_ML0710/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but SelectPercentile was fitted with feature names
  warnings.warn(


9 24.612777948379517


/opt/anaconda3/envs/NB_ML0710/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but SelectPercentile was fitted with feature names
  warnings.warn(


CPU times: user 60 s, sys: 9.83 s, total: 1min 9s
Wall time: 27.4 s


### XGBoost

- Because of computation time, we fixed max_depth as 5, and default settings were used for other parameters. 
- Class name have to be converted to integer for XGBoost

In [11]:
df_samples.loc[df_samples["class"]=="A", "class2"] = 0
df_samples.loc[df_samples["class"]=="B", "class2"] = 1
df_samples.loc[df_samples["class"]=="C", "class2"] = 2
df_samples.loc[df_samples["class"]=="D", "class2"] = 3
df_samples

,class,dataset,stage,MYCN,event/not,relapse/otherevent/not,alive/death,Event_Free_Survival_Time_in_Days,Overall_Survival_Time_in_Days,FirstEvent,diag_class,diag_age,class2
ID,,,,,,,,,,,,,
TARGET-30-PAIFXV-01A-01D,B,Target,4,N,Event,Event,D,630.0,1114.0,death of disease,1,2004,1.0
TARGET-30-PAIPGU-01A-01D,B,Target,4,N,Event,Death,D,436.0,436.0,death of disease,1,898,1.0
TARGET-30-PAISNS-01A-01D,B,Target,4,N,Event,Event,D,410.0,421.0,death of disease,1,1070,1.0
TARGET-30-PAITCI-01A-01D,B,Target,4,N,Event,Event,D,232.0,324.0,death of disease,1,728,1.0
TARGET-30-PAIVHE-01A-01D,B,Target,4,N,Event,Event,D,672.0,1605.0,death of disease,1,1123,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM3407024,D,Ackermann,1,N,Event,relapse/progression,D,NaN,NaN,relapse/progression,0,0,3.0
GSM3407025,B,Ackermann,4,N,Not,no event,A,268.0,268.0,no event,1,2786,1.0
GSM3407026,D,Ackermann,3,N,Not,no event,A,NaN,NaN,no event,0,323,3.0


In [12]:
%%time

start = time.time()

selPer_range = [10]

classes = {0:"A", 1:"B", 2:"C", 3:"D"}
df_res = pd.DataFrame()
df_imp = pd.DataFrame(index=df_beta.index)
for i in range(it):
    print(i, time.time()-start)
    (X_train, X_test ,y_train, y_test) = train_test_split(df_beta.transpose(), df_samples["class2"].to_list(), test_size = 0.3)
    # X_selected = SelectPercentile(score_func=chi2)
    selector = SelectPercentile(score_func=chi2, percentile=10) 
    selector.fit(X_train,y_train)
    X_selected = selector.transform(X_train)
    a = XGBClassifier(n_estimators=1000, max_depth=5,n_jobs=-1)
    a.fit(X_selected,y_train)

    pred_list = a.predict(selector.transform(X_test))
    prec_tmp,reca_tmp,f1_tmp,support_tmp = precision_recall_fscore_support(y_test, pred_list, average=None, labels=[0,1,2,3])

    df_tmp = pd.DataFrame(a.feature_importances_, columns=[i],index=selector.transform(pd.DataFrame(df_beta.index).T)[0])
    df_imp = df_imp.join(df_tmp)
    df_res.at[i, "mean_score"] = a.score(selector.transform(X_test), y_test)
    for j in classes.keys():
        df_res.at[i, classes[j]+"_precision"] = prec_tmp[j]
        df_res.at[i, classes[j]+"_recall"] = reca_tmp[j]
        df_res.at[i, classes[j]+"_f1"] = f1_tmp[j]
        df_res.at[i, classes[j]+"_support"] = support_tmp[j]

df_res.to_csv(outdir+"XGB.csv")
df_imp.to_csv(outdir+"XGB_imp.csv")

0 0.0008778572082519531


/opt/anaconda3/envs/NB_ML0710/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but SelectPercentile was fitted with feature names
  warnings.warn(


1 8.079022884368896


/opt/anaconda3/envs/NB_ML0710/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but SelectPercentile was fitted with feature names
  warnings.warn(


2 15.189906120300293


...

8 58.49655199050903


/opt/anaconda3/envs/NB_ML0710/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but SelectPercentile was fitted with feature names
  warnings.warn(


9 65.74954295158386


/opt/anaconda3/envs/NB_ML0710/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but SelectPercentile was fitted with feature names
  warnings.warn(


CPU times: user 9min 13s, sys: 7.52 s, total: 9min 20s
Wall time: 1min 13s


### lightGBM

In [13]:
%%time

start = time.time()
selPer_range = [10]

df_res = pd.DataFrame()
df_imp = pd.DataFrame(index=df_beta.index)
for i in range(it):
    print(i, time.time()-start)
    (X_train, X_test ,y_train, y_test) = train_test_split(df_beta.transpose(), df_samples["class"], test_size = 0.3)
    
    selector = SelectPercentile(score_func=chi2, percentile=10) 
    selector.fit(X_train,y_train)
    X_selected = selector.transform(X_train)
    a = LGBMClassifier(n_estimators=1000, max_depth=5,n_jobs=-1)
    a.fit(X_selected,y_train)

    classes = sorted(df_samples["class"].unique())
    pred_list = a.predict(selector.transform(X_test))
    prec_tmp,reca_tmp,f1_tmp,support_tmp = precision_recall_fscore_support(y_test, pred_list, average=None, labels=classes)

    df_tmp = pd.DataFrame(a.feature_importances_, columns=[i],index=selector.transform(pd.DataFrame(df_beta.index).T)[0])
    df_imp = df_imp.join(df_tmp)
    df_res.at[i, "mean_score"] = a.score(selector.transform(X_test), y_test)
    for j in range(len(classes)):
        df_res.at[i, classes[j]+"_precision"] = prec_tmp[j]
        df_res.at[i, classes[j]+"_recall"] = reca_tmp[j]
        df_res.at[i, classes[j]+"_f1"] = f1_tmp[j]
        df_res.at[i, classes[j]+"_support"] = support_tmp[j]
df_res.to_csv(outdir+"LGBM.csv")
df_imp.to_csv(outdir+"LGBM_imp.csv")

0 0.0007469654083251953


/opt/anaconda3/envs/NB_ML0710/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but SelectPercentile was fitted with feature names
  warnings.warn(


1 3.5707578659057617


/opt/anaconda3/envs/NB_ML0710/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but SelectPercentile was fitted with feature names
  warnings.warn(


2 6.910394191741943


...

8 27.34411120414734


/opt/anaconda3/envs/NB_ML0710/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but SelectPercentile was fitted with feature names
  warnings.warn(


9 30.78502607345581


/opt/anaconda3/envs/NB_ML0710/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but SelectPercentile was fitted with feature names
  warnings.warn(


CPU times: user 4min 16s, sys: 4.99 s, total: 4min 21s
Wall time: 34.2 s


### ExtraTrees

In [14]:
%%time

start = time.time()
selPer_range = [10]

df_res = pd.DataFrame()
df_imp = pd.DataFrame(index=df_beta.index)
for i in range(it):
    print(i, time.time()-start)
    (X_train, X_test ,y_train, y_test) = train_test_split(df_beta.transpose(), df_samples["class"], test_size = 0.3)

    selector = SelectPercentile(score_func=chi2, percentile=10) 
    selector.fit(X_train,y_train)
    X_selected = selector.transform(X_train)
    a = ExtraTreesClassifier(n_estimators=1000, max_depth=5, n_jobs=-1)
    a.fit(X_selected,y_train)

    classes = sorted(df_samples["class"].unique())
    pred_list = a.predict(selector.transform(X_test))
    prec_tmp,reca_tmp,f1_tmp,support_tmp = precision_recall_fscore_support(y_test, pred_list, average=None, labels=classes)

    df_tmp = pd.DataFrame(a.feature_importances_, columns=[i],index=selector.transform(pd.DataFrame(df_beta.index).T)[0])
    df_imp = df_imp.join(df_tmp)
    df_res.at[i, "mean_score"] = a.score(selector.transform(X_test), y_test)
    for j in range(len(classes)):
        df_res.at[i, classes[j]+"_precision"] = prec_tmp[j]
        df_res.at[i, classes[j]+"_recall"] = reca_tmp[j]
        df_res.at[i, classes[j]+"_f1"] = f1_tmp[j]
        df_res.at[i, classes[j]+"_support"] = support_tmp[j]
df_res.to_csv(outdir+"exTrees.csv")
df_imp.to_csv(outdir+"exTrees_imp.csv")

0 0.0007491111755371094


/opt/anaconda3/envs/NB_ML0710/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/NB_ML0710/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but SelectPercentile was fitted with feature names
  warnings.warn(


1 2.2499990463256836


/opt/anaconda3/envs/NB_ML0710/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/NB_ML0710/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but SelectPercentile was fitted with feature names
  warnings.warn(


2 4.452784061431885


...

8 17.617082118988037


/opt/anaconda3/envs/NB_ML0710/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/NB_ML0710/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but SelectPercentile was fitted with feature names
  warnings.warn(


9 19.814378023147583


/opt/anaconda3/envs/NB_ML0710/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/NB_ML0710/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but SelectPercentile was fitted with feature names
  warnings.warn(


CPU times: user 30.7 s, sys: 9.4 s, total: 40.1 s
Wall time: 22.1 s


# Compare scores

In [15]:
RF_score = pd.read_csv(outdir+"RF.csv", index_col=0)
XGB_score = pd.read_csv(outdir+"XGB.csv", index_col=0)
LGBM_score = pd.read_csv(outdir+"LGBM.csv", index_col=0)
exTrees_score = pd.read_csv(outdir+"exTrees.csv", index_col=0)
exTrees_score

,mean_score,A_precision,A_recall,A_f1,A_support,B_precision,B_recall,B_f1,B_support,C_precision,C_recall,C_f1,C_support,D_precision,D_recall,D_f1,D_support
0,0.520270,0.0,0.0,0.0,37.0,0.520270,1.0,0.684444,77.0,0.0,0.0,0.0,16.0,0.0,0.0,0.0,18.0
1,0.520270,0.0,0.0,0.0,39.0,0.520270,1.0,0.684444,77.0,0.0,0.0,0.0,11.0,0.0,0.0,0.0,21.0
2,0.432432,0.0,0.0,0.0,42.0,0.432432,1.0,0.603774,64.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,30.0
3,0.425676,0.0,0.0,0.0,30.0,0.425676,1.0,0.597156,63.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0,37.0
4,0.493243,0.0,0.0,0.0,35.0,0.493243,1.0,0.660633,73.0,0.0,0.0,0.0,14.0,0.0,0.0,0.0,26.0
5,0.486486,0.0,0.0,0.0,36.0,0.486486,1.0,0.654545,72.0,0.0,0.0,0.0,17.0,0.0,0.0,0.0,23.0
6,0.486486,0.0,0.0,0.0,32.0,0.486486,1.0,0.654545,72.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,24.0
7,0.527027,0.0,0.0,0.0,24.0,0.534247,1.0,0.696429,78.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0,28.0
8,0.472973,0.0,0.0,0.0,39.0,0.472973,1.0,0.642202,70.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,24.0
9,0.513514,0.0,0.0,0.0,35.0,0.517007,1.0,0.681614,76.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,22.0


In [16]:
for a in [RF_score, XGB_score, LGBM_score, exTrees_score]:
    print("precision")
    print(a["A_precision"].mean(),a["B_precision"].mean(),a["C_precision"].mean(),a["D_precision"].mean())
    print("recall")
    print(a["A_recall"].mean(),a["B_recall"].mean(),a["C_recall"].mean(),a["D_recall"].mean())    

precision
0.2597961249277038 0.46011355989212016 0.1783333333333333 0.095
recall
0.05800572662868485 0.8896086912381648 0.03696862348178137 0.011275132275132271
precision
0.24864023657501916 0.5091175138032688 0.19095238095238093 0.15863095238095237
recall
0.12544588332291756 0.8142050160582304 0.04821289821289819 0.038631780561654325
precision
0.20314232419091746 0.5020879296447365 0.06666666666666667 0.2795021645021645
recall
0.12329116740881445 0.7874494099105162 0.013247863247863239 0.07846924786579954
precision
0.0 0.48890912159013916 0.0 0.0
recall
0.0 1.0 0.0 0.0


In [17]:
df_res = pd.DataFrame()

classes = ["A", "B", "C", "D"]
classifier = ["RF", "XGB", "LGBM", "exTrees"]

for i1,a in enumerate([RF_score, XGB_score, LGBM_score, exTrees_score]):
    for k in classes:
        df_res.at["mean_score", classifier[i1]] = a["mean_score"].mean()
    for k in classes:
        df_res.at[k+"_precision", classifier[i1]] = a[k+"_precision"].mean()
    for k in classes:
        df_res.at[k+"_recall", classifier[i1]] = a[k+"_recall"].mean()
    for k in classes:
        df_res.at[k+"_f1", classifier[i1]] = a[k+"_f1"].mean()
df_res

,RF,XGB,LGBM,exTrees
mean_score,0.432432,0.450000,0.435811,0.487838
A_precision,0.259796,0.248640,0.203142,0.000000
B_precision,0.460114,0.509118,0.502088,0.488909
C_precision,0.178333,0.190952,0.066667,0.000000
D_precision,0.095000,0.158631,0.279502,0.000000
A_recall,0.058006,0.125446,0.123291,0.000000
B_recall,0.889609,0.814205,0.787449,1.000000
C_recall,0.036969,0.048213,0.013248,0.000000
D_recall,0.011275,0.038632,0.078469,0.000000
A_f1,0.090008,0.163663,0.152082,0.000000


In [18]:
RF_imp = pd.read_csv(outdir+"RF_imp.csv", index_col="ID_REF")
XGB_imp = pd.read_csv(outdir+"XGB_imp.csv", index_col="ID_REF")
LGBM_imp = pd.read_csv(outdir+"LGBM_imp.csv", index_col=0)
exTrees_imp = pd.read_csv(outdir+"exTrees_imp.csv", index_col="ID_REF")
exTrees_imp

,0,1,2,3,4,5,6,7,8,9
ID_REF,,,,,,,,,,
cg07881041,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cg18478105,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cg23229610,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cg03513874,NaN,0.000817,0.001339,NaN,NaN,0.000899,0.000827,NaN,NaN,0.000696
cg09835024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
cg04868734,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cg14069736,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cg03760226,NaN,NaN,0.001132,NaN,0.00028,NaN,NaN,NaN,NaN,NaN


In [19]:
print(
    RF_imp.loc[RF_imp.sum(axis=1)!=0].index.shape,
    XGB_imp.loc[XGB_imp.sum(axis=1)!=0].index.shape,
    LGBM_imp.loc[LGBM_imp.sum(axis=1)!=0].index.shape,
    exTrees_imp.loc[exTrees_imp.sum(axis=1)!=0].index.shape
     )

(2292,) (2036,) (2252,) (2309,)


In [20]:
num_genes = 15000
labels = venn.get_labels([
    set(RF_imp.sum(axis=1).sort_values(ascending=False).index[:num_genes]),
    set(XGB_imp.sum(axis=1).sort_values(ascending=False).index[:num_genes]),
    set(LGBM_imp.sum(axis=1).sort_values(ascending=False).index[:num_genes]),
    set(exTrees_imp.sum(axis=1).sort_values(ascending=False).index[:num_genes])], fill=['number'])
fig, ax = venn.venn4(labels, names=['RF', 'XGB', 'LGBM', 'exTrees'])
fig.show()

NameError: name 'venn' is not defined

In [ ]:
RF_imp.loc[RF_imp.sum(axis=1)!=0]
RF_imp.sum(axis=1).sort_values(ascending=False)

In [ ]:
df_imp_mat = pd.DataFrame(index=RF_imp.index)
df_imp_mat["RF"] = RF_imp.sum(axis=1)
df_imp_mat["XGB"] = XGB_imp.sum(axis=1)
df_imp_mat["LGBM"] = LGBM_imp.sum(axis=1)
df_imp_mat["exTrees"] = exTrees_imp.sum(axis=1)
df_imp_mat = df_imp_mat.fillna(0)
df_imp_mat

In [ ]:
df_imp_mat = df_imp_mat.loc[df_imp_mat.sum(axis=1)!=0]
df_imp_mat.corr()
df_imp_mat.corr().at["RF","XGB"]

In [ ]:
df_imp_mat

In [ ]:
%%time

df_corr_tmp = pd.DataFrame()
for i1 in range(3):
    for i2 in range(i1+1,4):
        k = df_imp_mat.columns[i1]+"-"+df_imp_mat.columns[i2]
        df_corr_tmp[k] = [pd.DataFrame(np.array([df_imp_mat["RF"].sample(df_imp_mat.shape[0]),df_imp_mat["XGB"]]).T).corr().at[0,1] for i in range(1000)]
        
        print(df_imp_mat.columns[i1], df_imp_mat.columns[i2], df_corr_tmp.loc[abs(df_corr_tmp[k])> df_imp_mat.corr().at[df_imp_mat.columns[i1],df_imp_mat.columns[i2]]].shape[0])
df_corr_tmp


In [ ]:
fig = plt.figure(figsize=(20,20))

i=1
for i1 in range(4):
    for i2 in range(4):
        print(df_imp_mat.columns[i1],df_imp_mat.columns[i2])
        ax = fig.add_subplot(4,4,i)
        ax = sns.regplot(data=df_imp_mat, x=df_imp_mat.columns[i1], y=df_imp_mat.columns[i2])        
        i +=1

# sns.regplot(data=df_imp_mat, x="RF", y="XGB")

In [ ]:
XGB_imp.sum(axis=1).sort_values()